In [2]:
import copy
import math
import numpy as np
import xarray as xr
import pandas as pd
from glob import glob

# READING PM2.5 DATA FILES

Chị Thon hãy để những file `.nc` trong thư mục là `data` nhé!

In [5]:
get_year = lambda file_path: int(file_path.split('.')[-2][:4])


def read_pmi_asia(file_path):
    ncin = xr.open_dataset(file_path)

    year = get_year(file_path)

    # Extract latitude and longitude
    lat = ncin.variables['lat'][:]
    lon = ncin.variables['lon'][:]

    # Extract the last variable
    pm25 = ncin.variables[list(ncin.variables.keys())[-1]][:]

    # Create xarray DataArray objects
    pm25_da = xr.DataArray(pm25, dims=('lat', 'lon'), coords={'lat': lat, 'lon': lon})
    lat_da = xr.DataArray(lat, dims=('lat',), coords={'lat': lat})
    lon_da = xr.DataArray(lon, dims=('lon',), coords={'lon': lon})

    # Create an xarray Dataset
    data = xr.Dataset({'pm25': pm25_da, 'lat': lat_da, 'lon': lon_da}).to_dataframe()

    # Reset index to convert MultiIndex to regular columns
    data = data.reset_index()

    # clean data - remove NA
    data = data.dropna()

    return data, year


file_paths = glob('data/*.nc')
file_paths.sort()
file_paths

['data/V5GL04.HybridPM25.Asia.201501-201512.nc']

## Read Location of Villages

In [110]:
def read_vil_data(path="VN_geo.xlsx"):
    vil_data = pd.read_excel(path)  # vil = village
    vil_data = vil_data.rename(columns={"Unnamed: 0": "num"})
    return vil_data


vil_data = read_vil_data()
vil_data.head()

,num,name,max_lat,min_lat,max_lon,min_lon
0,0,Đà Nẵng_Cẩm Lệ_Hòa Phát,16.061979,16.015436,108.198692,108.159225
1,1,Đà Nẵng_Cẩm Lệ_Hòa Xuân,16.028502,15.971738,108.248512,108.208954
2,2,Đà Nẵng_Cẩm Lệ_Khuê Trung,16.040258,16.008497,108.224228,108.197716
3,3,Đà Nẵng_Hải Châu_Bình Hiên,16.061552,16.055838,108.226624,108.212067
4,4,Đà Nẵng_Hải Châu_Bình Thuận,16.057447,16.052580,108.226715,108.211433


## Extract PMI for each Village in All Years

In [114]:
# convert lat and lon to index
def get_idx(min_value, value, interval=0.01):
    return int(np.round((value - min_value) / interval))


# extract subarea containing the village
def extract_subarray(data, x1, x2, y1, y2):
    if x1 == x2 and y1 == y2:
        return data[x1, y1]
    elif x1 < x2 and y1 == y2:
        return data[x1:x2 + 1, y1]
    elif x1 == x2 and y1 < y2:
        return data[x1, y1:y2 + 1]
    elif x1 < x2 and y1 < y2:
        return data[x1:x2 + 1, y1:y2 + 1]
    else:
        raise ValueError("Invalid indices")


def get_pmi_vn(pmi_asia, year, vil_data, interval=0.01):
    vil_data_pmi_year = copy.deepcopy(vil_data)

    min_vn_lat = vil_data_pmi_year.min_lat.min() - interval * 50
    max_vn_lat = vil_data_pmi_year.max_lat.max() + interval * 50
    min_vn_lon = vil_data_pmi_year.min_lon.min() - interval * 50
    max_vn_lon = vil_data_pmi_year.max_lon.max() + interval * 50

    # dataframe
    pmi_df = pmi_asia[(pmi_asia.lat >= min_vn_lat) & (pmi_asia.lat <= max_vn_lat) & (pmi_asia.lon >= min_vn_lon) & (pmi_asia.lon <= max_vn_lon)]

    # Build a np array for the PMI data in VN
    num_lat, num_lon = math.ceil((max_vn_lat - min_vn_lat) / interval), math.ceil((max_vn_lon - min_vn_lon) / interval)
    pmi_vn = np.zeros((num_lat, num_lon)) + 10000

    # Calculate the indices for the PMI data in the array
    lat_indices = ((pmi_df.lat - min_vn_lat) / interval).astype(int)
    lon_indices = ((pmi_df.lon - min_vn_lon) / interval).astype(int)

    # Update the PMI data array using indexing
    pmi_vn[lat_indices, lon_indices] = pmi_df.pm25.values

    for vil_index, vil in vil_data_pmi_year.iterrows():
        low_lat_idx = get_idx(min_vn_lat, vil.min_lat)
        low_lon_idx = get_idx(min_vn_lon, vil.min_lon)

        high_lat_idx = get_idx(min_vn_lat, vil.max_lat)
        high_lon_idx = get_idx(min_vn_lon, vil.max_lon)

        while True:
            pmi_area = extract_subarray(pmi_vn, low_lat_idx, high_lat_idx, low_lon_idx, high_lon_idx)
            pmi_mask = pmi_area < 10000

            if pmi_mask.sum() > 0:
                vil_data_pmi_year.loc[vil_index, 'pm25'] = pmi_area[pmi_mask].mean()
                vil_data_pmi_year.loc[vil_index, 'year'] = str(year)  # save year as string
                break
            else:
                low_lat_idx = max(0, low_lat_idx - 1)
                low_lon_idx = max(0, low_lon_idx - 1)
                high_lat_idx = min(pmi_vn.shape[0] - 1, high_lat_idx + 1)
                high_lon_idx = min(pmi_vn.shape[1] - 1, high_lon_idx + 1)
    return vil_data_pmi_year

In [127]:
all_vil_data_pmi = []

# Read PMI data of Asia over the years and extract PMI data of Vietnam
for i, file_path in enumerate(file_paths):
    print(f'Processing file {i + 1}/{len(file_paths)}.')

    # Read PMI data of Asia and the year
    pmi_asia, year = read_pmi_asia(file_path)

    # Extract PMI data of Vietnam in the year
    vil_data_pmi_year = get_pmi_vn(pmi_asia, year, vil_data, interval=0.01)

    # Append the PMI data of Vietnam in the year to the list
    all_vil_data_pmi.append(vil_data_pmi_year)

# Concatenate the PMI data of Vietnam over the years
all_vil_data_pmi = pd.concat(all_vil_data_pmi)

Processing file 1/2.
Processing file 2/2.


In [124]:
all_vil_data_pmi.head()

,num,name,max_lat,min_lat,max_lon,min_lon,pm25,year
0,0,Đà Nẵng_Cẩm Lệ_Hòa Phát,16.061979,16.015436,108.198692,108.159225,18.256667,2015
1,1,Đà Nẵng_Cẩm Lệ_Hòa Xuân,16.028502,15.971738,108.248512,108.208954,18.271429,2015
2,2,Đà Nẵng_Cẩm Lệ_Khuê Trung,16.040258,16.008497,108.224228,108.197716,17.983333,2015
3,3,Đà Nẵng_Hải Châu_Bình Hiên,16.061552,16.055838,108.226624,108.212067,18.500000,2015
4,4,Đà Nẵng_Hải Châu_Bình Thuận,16.057447,16.052580,108.226715,108.211433,17.900000,2015


In [125]:
all_vil_data_pmi.tail()

,num,name,max_lat,min_lat,max_lon,min_lon,pm25,year
10771,10771,Yên Bái_Yên Bình_VÜnh Kiên,21.799368,21.746712,105.058495,104.979546,40.358333,2016
10772,10772,Yên Bái_Yên Bình_Xuân Lai,21.937389,21.896063,105.022125,104.918839,38.524528,2016
10773,10773,Yên Bái_Yên Bình_Xuân Long,22.072737,21.974617,104.962059,104.860825,37.765657,2016
10774,10774,Yên Bái_Yên Bình_Yên Bình,21.795162,21.717907,105.080101,104.931320,40.866418,2016
10775,10775,Yên Bái_Yên Bình_Yên Thành,21.904144,21.851635,105.043488,104.923706,38.618055,2016


In [126]:
# save to excel file
all_vil_data_pmi.to_excel("village_pmi_all_years.xlsx", index=False)

<ipython-input-126-1eea54ba533e>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  all_vil_data_pmi.to_excel("village_pmi_all_years.xlsx", index=False)
